# Untitled Kununu Project
<b>Project Goal:</b> The automatic analysis of employer evaluations from the kununu portal. Although the code shall be scalable and flexible, it shall at this moment focuses on:<br>
- One Company, and<br>
- German Language Reviews<br>
- The Feature "Recomandation".<br><br>

<b>Strategic Goal:</b> The strategic goal pursued by this Project is to:
- excercise, and<br>
- flex<br>

my Data Science muscles.<br>

<span style="color: red;">
<b>Next Steps</b><br>
- Find Pun for Project-Title<br>
- Write a Readme<br>
- Add Pictures/ Examples/ Comments to guide Reader through Notebook<br>
- Get Peer Feedback<br>
</span>

## 00 : Set Up Workshop

### 00-01 : Import Libraries
<span style="color: red;">
<b>Next Steps</b><br>
- Consider: https://stackoverflow.com/questions/43377265/determine-if-text-is-in-english/48436520#48436520<br>
</span>

In [ ]:
# Vectors, Matrices and thier Math
import numpy as np
# DataFrames
import pandas as pd
# Working with HTML Files
from lxml import html
# Turn HTML Element into string
from lxml.etree import tostring
# Basic Visualizations
import matplotlib.pyplot as plt
# Fancy Visualizations
import seaborn as sns
# Identify Language
from langdetect import detect
# Natural Language Handling
import nltk
# Stopwords
from nltk.corpus import stopwords
# Tokenize Words
from nltk import word_tokenize
# How often does which token appear in a list
from collections import Counter
# Regular Expressions
import re

### 00-02 : Get additional Data for Libraries

In [ ]:
# Download Stopwords
nltk.download('stopwords')

### 00-03 : Homemade Functions
<br>
<span style="color: red;">
<b>Next Steps</b><br>
- Polish Code transform function<br>
- Functions that are only used in one Chapter shall be defined there<br>
</span>

In [ ]:
# Takes evaluation-string and turns it into a Dataframe with the right format
def transform(review):
    # Set up Collector for Results
    collector_grades = pd.DataFrame([])
    # Loop through Categories within review
    for category in review:
        # Extract Title of Category
        title = category[0]
        # Extract written Text; if empty, turn to missing value
        text  = category[1]
        if text == []:
            #text = np.nan    
            text = None
        # Extract Number of Stars
        grade = category[2]
        # Shape information into DataFrame        
        result = pd.DataFrame([text,grade]).T
        # Name the Columns ("_T" short for text, "_N" for number)
        result.columns = [str(title+"_N"),str(title+"_T")]
        # Drop empty columns
        result.dropna(axis = 1, how = 'any', thresh = None, subset = None, inplace = True)        
        # Join with allready collected evaluation from this review
        collector_grades = pd.concat([collector_grades, result], axis = 1)
    # Return results
    return(collector_grades)

In [ ]:
# Takes requested result out of the table with the data-reconaissance results and turns it into a HTML expression
#(IOT save time by not typing the whole expression)
def xpr(tag):
    cat  = recon.loc[tag].type
    code = recon.loc[tag].code
    text = recon.loc[tag].text
    # Takes Category and code and turns it into a string with the right format
    result = str('//'+cat+'[@class=\"'+code+'\"]')
    # If requested (text == 1), the code to extract the text will be added
    if text == 1:
        result += "/text()"
    # Return the Results
    return(result)

### 00-04 : Settings

In [ ]:
# Define Color Scheme
flatui = ["lightgreen", 'tomato']
knncol = sns.color_palette(flatui)

# 01 : Data Mining
<br>
<span style="color: red;">
<b>Next Steps</b><br>
- Scrape directly from WebPage<br>    
</span>

In [ ]:
with open(r'kununu3.html', "r", encoding='utf-8') as f:
    page = f.read()
raw_html = html.fromstring(page)

## 02 : Data Cleaning
<br>
<span style="color: red;">
<b>Next Steps</b><br>
- What does "grades.values.tolist()" do again?!<br>
- inputformat Text (unicode)<br>
- Feature Formats<br>
- Ferature Name/ Dictionary<br>
- Comment Code<br>
- Polish Code<br>
</span>

### 02-01 : Data Reconnaissance
<br>
<span style="color: red;">
<b>Next Steps</b><br>
- Write Prose about the findings, leading to the Results in the DataFrame<br>
</span>

After exploring the Data Source - which is in HTML - I found the necessary tags to extract the Data. I turn these into a DataFrame, so I can use the tags (utilizing the xpr-function from 00-03) to extract the data.

In [ ]:
# Enter Results from Data Reconnaissance into a DataFrame
recon = pd.DataFrame([['article','index__contentBlock__7vKo-',0],
                      ['h3','index__title__2uQec',1],
                      ['span','index__score__16yy9',1],
                      ['span','index__recommendation__jftd3',1],
                      ['span','index__position__mCyeO',1],
                      ['span','index__sentence__3PKUg index__middot__3vlu3',1],
                      ['div','index__factor__3Z15R index__factor__3YJYJ index__expanded__349V7',0],
                      ['h4','index__title__W4hOp',1],
                      ['p','index__plainText__lgNCM',1],
                      ['span','index__stars__2ads4 index__medium__1wpWb index__stars__3lgvx',0],
                      ['div','index__factor__3Z15R',0]])
# Name Features
recon.columns =["type","code","text"]
# Name the Tags
recon.index = ['review','headline','score','recom','status','department',
               'cat_one','cat_one_title','cat_one_text','cat_one_grade',
               'cat_two']
# Show Result
recon

In [ ]:
xpr('cat_one_grade')

### 02-02 : Shape the Data into DataFrame
<br>
<span style="color: red;">
<b>Next Steps</b><br>
- Combine Tag One and Tag Two Loop<br>
    
- Extract Time (    # Extract the Time the Review was given
   
    
    try:
        # If time is given
        time = [''.join(review_html.xpath(xpr('time','index__date__37sgB',0))[0].attrib.get('datetime'))]
       except:
        # If not, save 0
        time = '0'    )
    
- utilize list expressions<br>
- improve performance (!!!)<br>
</span>

In [ ]:
# Set up Collector for overall result
overview = pd.DataFrame([])

# Loop Through Reviews in the HTML code
for idx in raw_html.xpath(xpr('review')):
    # Select individual review 
    review_html = html.fromstring(tostring(idx))
    
    # Extract the Headline from the Review
    headline   = review_html.xpath(xpr('headline'))
    # Extract the Average Score
    score      = review_html.xpath(xpr('score'))
        
    # Extract the Recommendation
    recom      = review_html.xpath(xpr('recom'))    
    # Extract the Status of the reviewer   
    status     = [''.join(review_html.xpath(xpr('status')))]
    # Extract the department of the reviewer      
    department = review_html.xpath(xpr('department'))    

    # Set up Collector for Categories within the review   
    evaluation = pd.DataFrame([])
    
    # Review the categories Tag One
    categories_one = review_html.xpath(xpr('cat_one'))    
    # Loop through these categories
    for idx in categories_one:
        # Select category
        category      = html.fromstring(tostring(idx))
        # Extract Title of Category
        cat_one_title = str(category.xpath(xpr('cat_one_title'))[0])
        # Extract Evaluation Text of Category        
        cat_one_text  = str(category.xpath(xpr('cat_one_text')))             
        # Change Missing Value Format
        if cat_one_text == '[]':
            cat_one_text = np.nan   
        
        # Try to extract grade, if it extists           
        try:
            cat_one_grade = int(category.xpath(xpr('cat_one_grade'))[0].attrib.get('data-score'))
        except:
            cat_one_grade = np.nan
        # Turn Results into DataFrame
        summary = pd.DataFrame([cat_one_title, cat_one_grade, cat_one_text]).T    
        # Add to the List
        evaluation = evaluation.append(summary) 
        
    # Review the categories Tag Two
    categories_two = review_html.xpath(xpr('cat_two'))          
    for idx in categories_two:
        category     = html.fromstring(tostring(idx))
        # Extract Title of Category
        cat_two_title = str(category.xpath(xpr('cat_one_title'))[0])
        # Extract Evaluation Text of Category        
        cat_two_text  = str(category.xpath(xpr('cat_one_text'))[0])               
        # Change Missing Value Format
        if cat_two_text == '[]':
            cat_two_text = np.nan           
       
        # Try to extract grade, if it extists           
        try:
            cat_two_grade = int(category.xpath(xpr('cat_one_grade'))[0].attrib.get('data-score'))
        except:
            cat_two_grade = np.nan
        # Turn Results into DataFrame
        summary = pd.DataFrame([cat_two_title, cat_two_grade, cat_two_text]).T    
        # Add to the List
        evaluation = evaluation.append(summary)        
        
    # Reset Index of collected evaluations
    evaluation.reset_index(drop = True, inplace = True)
    # and turn into list for easy handling
    evaluation = evaluation.values.tolist()
    
    # Put extracted Information from review into DataFrame format; score turned to list
    summary = pd.DataFrame([headline, 
                             [score], recom, status, department, 
                             pd.Series([evaluation])]).T
    # and add it to the others
    overview = overview.append(summary)

# Reset Index
overview.reset_index(drop = True, inplace = True)
# IOT drop first review (a dummy)
overview.drop([0], inplace = True)
# and finalize the index
overview.reset_index(drop = True, inplace = True)
# name the features
overview.columns = ["headline","avg_score","recom","status","depart","evaluations"]

In [ ]:
# Set up Collector for results
collector_reviews = pd.DataFrame([])
# loop through reviews
for review in overview.evaluations:
    # If existing ...  
    if(review != None):
        # change into DataFrame using 00-03's transform function and add to other results
        collector_reviews = pd.concat([collector_reviews, 
                                       transform(review)],
                                      axis = 0, 
                                      sort = False,
                                      join = 'outer')
# reset index
collector_reviews.reset_index(drop = True, inplace = True)

# Attach to newly formatted to core DataFrame, and
overview = pd.concat([overview, collector_reviews], axis=1)
# drop original feature
overview.drop('evaluations', axis = 1, inplace = True)

### 02-03 : Change Feature Formats
<br>
<span style="color: red;">
<b>Next Steps</b><br>
- Integrate this into 02-02<br>
- time pd.to_datetime(overview.date)<br>
</span>

In [ ]:
overview.headline  = overview.headline.astype(str)
overview.recom     = overview.recom.astype(str)
overview.status    = overview.status.astype(str)
overview.depart    = overview.depart.astype(str)
for idx in [s for s in overview.columns[6:] if "_T" in s]:
    overview[idx] = overview[idx].astype(str)
for idx in [s for s in overview.columns[6:] if "_N" in s]:
    overview[idx] = overview[idx].replace(',','.').astype(float)

In [ ]:
overview.avg_score = [float(idx[0].replace(',','.')) for idx in overview.avg_score]

### 02-04 : Extract additional Information from Features
<br>
<span style="color: red;">
<b>Next Steps</b><br>
- Change into categorical feature if applicable<br>
</span>

#### 02-04-01 : Status -> Current or Former Employee

In [ ]:
overview["ex"] = overview.status.str.contains("Ex-", regex = False)
overview["status"] = overview.status.str.replace("Ex-", "")

#### 02-04-02 : Department -> Location, Department, Division, Time, Vocational Training
<br>
<span style="color: red;">
<b>Next Steps</b><br>
- Add to Data Reconaissance<br>
- Remove the \n earlier<br>
- Polish and Comment Code<br>
- Work with Regex/ str.split to make this safe<br>
- Extract Time of Review in 02-02, use here to get time<br>
</span>

In [ ]:
def save_check(sublist,stringz):
    try:
        return(sublist.index(stringz))+1
    except:
        return(None)

In [ ]:
def save_take(sublist,number):
    if number != None:
        return(sublist[number])
    else:
        return(None)

In [ ]:
# Remove \n
overview.depart = overview.depart.str.replace('\n','')

In [ ]:
# Tokenize IAW results from Data Reconaissance
deptoken = [re.split(r'( zum/zur )|( bis )|( bei )|( in )|( im Bereich )',idx) for idx in overview.depart]
# Remove Nones
deptoken = [[i for i in sublist if i]
         for sublist in deptoken]

In [ ]:
# Extract Location (after the ' in ' bracket)
location = [sublist[sublist.index(' in ')+1] for sublist in deptoken]
# Remove everything that is not the location
location = [idx.replace(" gearbeitet.",'') for idx in location]
location = [idx.replace(" abgeschlossen.",'') for idx in location]
location = [idx.replace(" absolviert.",'') for idx in location]
# Add to Main DataFrame
overview['location'] = location

In [ ]:
# Extract Division (after the ' bei ' bracket)
division = [sublist[sublist.index(' bei ')+1] for sublist in deptoken]
overview['division'] = division

In [ ]:
# Extract Department
delta = [save_check(sublist,' im Bereich ') for sublist in deptoken]
department = [save_take(deptoken[idx], delta[idx]) for idx in range(0, len(overview.depart))]
overview["department"] = department

#### 02-04-03 : Review-Texts -> Language
<br>
<span style="color: red;">
<b>Next Steps</b><br>
- Polish this<br>
- What happens if all values are missing?<br>
- text_feats earlier<br>
</span>

In [ ]:
def safe_detect(string):
    try:
        return(detect(string))
    except:
        return('UKN')

In [ ]:
# All text Features
text_feats = ['headline']
text_feats += [idx for idx in overview.columns
if '_T' in idx]
# Add all written text
text = [[overview[feat][review] for feat in text_feats
if overview[feat][review] != 'nan'] for review in range(0,overview.shape[0])] 
# Join all text
text = [" ".join(idx) for idx in text]
# Add to DataFrame
overview['text'] = text
# Delete '\n'
overview['text'] = overview['text'].str.replace('\n',' ')

In [ ]:
overview['language'] = [safe_detect(idx) for idx in overview.text]

### 02-04 : Drop irrelevant Data
<i> Since this project focuses on the recommendation, all observations without one will be dropped.</i>

In [ ]:
overview.drop(overview[overview.recom == 'None'].index, inplace = True)
overview = overview.reset_index(drop = True)

<b>Warning!</b> Once the the Scope of this Project is extended and the observations are not dropped anymore - the code under 02-02 needs to be altered. At earlier times, when kununu did not have the recommendation feature, the ratings where tagged differently. The current choice of scope convinently circumvents this issue.</i>

## 03 : Analysis
<br>
<span style="color: red;">
<b>Next Steps</b><br>
- Automatic Identification of "Grading-Curve"'s (Unsupervised Learning: DBSCAN?)<br>
- Extract Time Feature<br>
- Make, and write down, conclusions<br>
- unify design/ colorscheme<br>
</span>
<br>
<i> Focus of this Project is the reviewer's recommendation. Every piece of Data is seen through this lense therefore. The analysis of each single features by itself is therfore skipped. Exception is the Guest of Honor itself: The Recommendation:</i> 

In [ ]:
plt.pie(overview.recom.value_counts(),labels = overview.recom.value_counts().index, autopct='%1.1f%%', colors = flatui)
plt.title("Recommendation")
plt.show()

### 03-01 : Numerical Features

#### 03-01-01 : Average Review Score

In [ ]:
bins = np.linspace(1,5,10)
sns.distplot(overview.avg_score[overview.recom=="Empfohlen"] , color="lightgreen", label="Recommended", kde = False, bins = bins)
sns.distplot(overview.avg_score[overview.recom=="Nicht empfohlen"] , color="tomato", label="Not Recommended", kde = False, bins = bins)
plt.xlabel("Average Score")
plt.ylabel("Number of Reviews")
plt.title("Kununu Reviews")
plt.legend()
plt.show()

#### 03-01-02 : Categories

In [ ]:
bins = np.linspace(0.5,5.5,6)
for idx in (overview.columns[overview.dtypes == float].drop("avg_score")):
    plt.title(str(idx))
    data = overview[idx]
    sns.distplot(data[overview.recom=="Empfohlen"] , color="lightgreen", label="Recommended", kde = False, bins = bins)
    sns.distplot(data[overview.recom=="Nicht empfohlen"] , color="tomato", label="Not Recommended", kde = False, bins = bins)
    plt.xlabel("Average Score")
    plt.ylabel("Number of Reviews")
    plt.title(idx.replace("_N",""))
    plt.legend()
    plt.show()

### 03-02 : Categorical Features
<br>
<span style="color: red;">
<b>Next Steps</b><br>
- Auto Combine Small/ Irrelevant<br>
- Colorscheme for Piecharts<br>
- Argh. Red/ Green switches!!<br>
- Make Functions out of this<br>
</span>
<br>

#### 03-02-01 : Former Worker / Current Worker
<br>
<span style="color: red;">
<b>Next Steps</b><br>
- Phrase Better<br>
</span>

In [ ]:
plt.pie(overview.ex.value_counts(),labels = overview.ex.value_counts().index, autopct='%1.1f%%')
plt.title("Reviews by Former Employees vs Current Employes")
plt.show()

In [ ]:
sns.catplot("recom", data = overview, kind = "count", col="ex", palette = knncol, alpha = 0.5)
plt.show()

#### 03-02-02 : Type of Contract

In [ ]:
plt.pie(overview.status.value_counts(),labels = overview.status.value_counts().index, autopct='%1.1f%%')
plt.title("Reviews by Type of Contract")
plt.show()

In [ ]:
sns.catplot("recom", data = overview, kind = "count", col="status", palette = knncol, col_wrap=2, alpha = 0.5)
plt.show()

#### 03-02-03 : Location

In [ ]:
plt.pie(overview.location.value_counts(),labels = overview.location.value_counts().index, autopct='%1.1f%%')
plt.title("Reviews by Location")
plt.show()

In [ ]:
sns.catplot("recom", data = overview, kind = "count", col="location", palette = knncol, col_wrap=2, alpha = 0.5)
plt.show()

#### 03-02-04 : Division

In [ ]:
plt.pie(overview.division.value_counts(),labels = overview.division.value_counts().index, autopct='%1.1f%%')
plt.title("Reviews by Division")
plt.show()

In [ ]:
sns.catplot("recom", data = overview, kind = "count", col="division", palette = knncol, col_wrap=2, alpha = 0.5)
plt.show()

#### 03-02-05 : Department

In [ ]:
plt.pie(overview.department.value_counts(),labels = overview.department.value_counts().index, autopct='%1.1f%%')
plt.title("Reviews by Department")
plt.show()

In [ ]:
sns.catplot("recom", data = overview, kind = "count", col="department", palette = knncol, col_wrap=2, alpha = 0.5)
plt.show()

#### 03-02-06 : Language
Not part of Text Analysis

In [ ]:
plt.pie(overview.language.value_counts(),labels = overview.language.value_counts().index, autopct='%1.1f%%')
plt.show()

<i> langdetect is not perfect - but it can only work with what it is fed. I am highly certain that the 'de' and 'en' are correct - and only those</i>

In [ ]:
sns.catplot("recom", data = overview, kind = "count", col="language", palette = knncol, col_wrap=2, alpha = 0.5)
plt.show()

### 03-02 : Textual Analysis

#### 03-02-01 : Preprocess Text

In [ ]:
# Delete non German observation
overview = overview[overview.language == 'de']
overview.reset_index(drop = True, inplace = True)
overview.text[10]

In [ ]:
# Tokenize Words
tokenized = [word_tokenize(idx) for idx in overview.text]
tokenized[10]

In [ ]:
# Leave numbers and punctuation behind
tokenized = [[idx_2 for idx_2 in idx
              if idx_2.isalpha()]
             for idx in tokenized]
tokenized[10]

In [ ]:
# All lowercase
tokenized = [[idx_2.lower() for idx_2 in idx]
             for idx in tokenized]
tokenized[10]

In [ ]:
# Take out Stopwords
tokenized = [[idx_2 for idx_2 in idx
              if idx_2 not in stopwords.words('german')]
             for idx in tokenized]
tokenized[10]

#### 03-02-02 : Bag of Words
<br>
<span style="color: red;">
<b>Next Steps</b><br>
- Rethink how to handle categories like "Gut am Arbeitgeber finde ich" / "Verbesserungsvorschläge" -> Employee Sentiment is in the open here<br>
- Apply Seaborn<br>
- COMMENT!<br>
</span>

##### 03-02-02-01 : All Text
Look at all the company's reviews as one.

In [ ]:
# Combine all Tokens
allwords = [val for sublist in tokenized for val in sublist]

In [ ]:
allwords_Counter = Counter(allwords)

In [ ]:
plt.barh(y = [idx[0] for idx in allwords_Counter.most_common(20)],
         width = np.array([idx[1] for idx in allwords_Counter.most_common(20)])/len(allwords))
plt.xticks(rotation = 90)
plt.title("Token Frequency, all Text")
plt.show()

##### 03-02-02-02 : By Recommandation

In [ ]:
minus_texts = [tokenized[idx] for idx in range(0,len(tokenized))
 if overview.recom[idx] == "Nicht empfohlen"]
minus_words = [val for sublist in minus_texts for val in sublist]
minus_count = Counter(minus_words).most_common(20)
plt.barh(y = [idx[0] for idx in minus_count], width = np.array([idx[1] for idx in minus_count])/len(minus_words), color = "tomato")
plt.xticks(rotation = 90)
plt.title("Word Frequency, Not Recommended")
plt.show()

In [ ]:
plus_texts = [tokenized[idx] for idx in range(0,len(tokenized))
 if overview.recom[idx] == "Empfohlen"]
plus_words = [val for sublist in plus_texts for val in sublist]
plus_count = Counter(plus_words).most_common(20)
plt.barh(y = [idx[0] for idx in plus_count], width = np.array([idx[1] for idx in plus_count])/len(plus_words), color = "lightgreen")
plt.xticks(rotation = 90)
plt.title("Word Frequency, Recommended")
plt.show()

In [ ]:
allall = pd.DataFrame.from_dict(Counter(allwords), orient='index')/len(allwords)
plus = pd.DataFrame.from_dict(Counter(plus_words), orient='index')/len(plus_words)
minus = pd.DataFrame.from_dict(Counter(minus_words), orient='index')/len(minus_words)

In [ ]:
collector_reviews = pd.concat([allall.T, 
                               minus.T],
                              axis = 0, 
                              sort = False,
                              join = 'outer')
collector_reviews = pd.concat([collector_reviews, 
                               plus.T],
                              axis = 0, 
                              sort = False,
                              join = 'outer')

collector_reviews = collector_reviews.fillna(0)
collector_reviews = collector_reviews.astype(float)
collector_reviews.reset_index(drop = True, inplace = True)

In [ ]:
plus_topflop = pd.DataFrame(collector_reviews.loc[2] - collector_reviews.loc[0]).sort_values(by = 0).head(10)
plus_topflop = plus_topflop.append(pd.DataFrame(collector_reviews.loc[2] -collector_reviews.loc[0]).sort_values(by = 0).tail(10))

In [ ]:
plt.barh(y = plus_topflop.index, width = plus_topflop[0], color = "lightgreen")
plt.xticks(rotation = 90)
plt.title("Top/ Flop 10: relative Word Frequency, Recommended")
plt.show()

In [ ]:
minus_topflop = pd.DataFrame(collector_reviews.loc[1] - collector_reviews.loc[0]).sort_values(by = 0).head(10)
minus_topflop = minus_topflop.append(pd.DataFrame(collector_reviews.loc[1] -collector_reviews.loc[0]).sort_values(by = 0).tail(10))

In [ ]:
plt.barh(y = minus_topflop.index, width = minus_topflop[0], color = 'tomato')
plt.xticks(rotation = 90)
plt.title("Top/ Flop 10: relative Word Frequency, Not Recommended")
plt.show()

## YY - Construction Site starts here

#### 03-02-05 : TFIDF Model

In [ ]:
import gensim
from gensim.corpora.dictionary import Dictionary
from gensim.models import TfidfModel

In [ ]:
dct = Dictionary(tokenized)

In [ ]:
corpus = [dct.doc2bow(line) for line in tokenized]

In [ ]:
model = TfidfModel(corpus)

In [ ]:
vector = model[corpus[0]]

In [ ]:
weights = [model[corpus[idx]] for idx in range(0,len(corpus))]

In [ ]:
flat_list = [item for sublist in weights for item in sublist]

In [ ]:
a = [len(idx) for idx in weights]
b = [idx for idx in range(0,len(weights))]

In [ ]:
taser = pd.DataFrame(np.repeat(b, a))

In [ ]:
taser.columns = ["text"]

In [ ]:
taser[]

In [ ]:
taser = pd.DataFrame(flat_list)

In [ ]:
taser["text"] = pd.DataFrame(np.repeat(b, a))

In [ ]:
taser.columns = ["id","weight","text"]

In [ ]:
lowest = min(taser.groupby("text").max().weight)

In [ ]:
sum(taser.weight >= lowest) / len(taser.weight)

In [ ]:
plt.hist(taser.weight)

In [ ]:
test = Counter([dct.get(idx) for idx in taser[taser.weight>lowest].reset_index(drop = True).id]).most_common(20)

In [ ]:
plt.barh(y = [idx[0] for idx in test], width = np.array([idx[1] for idx in test]))
plt.xticks(rotation = 90)
plt.title("Word Frequency, all Text")
plt.show()

In [ ]:
# Create the pie chart
plt.pie(values, labels=labels, autopct='%1.1f%%', startangle=140)

## ZZ - Notes

In [ ]:
import requests
page = requests.get('http://econpy.pythonanywhere.com/ex/001.html')
tree = html.fromstring(page.content)
#This will create a list of buyers:
buyers = tree.xpath('//div[@title="buyer-name"]/text()')
#This will create a list of prices
prices = tree.xpath('//span[@class="item-price"]/text()')

In [ ]:
temp = game.xpath('.//span[contains(@class, "platform_img")]')
# !! CONTAINS

In [ ]:
unternehmenslink anwählen
- Anzahl der Bewertungen einlesen
- Durch 10 Teilen = XYRPP
-- einlesen der daten

In [ ]:
https://python-graph-gallery.com/100-calling-a-color-with-seaborn/

In [ ]:
def corrlist(data):
    c = data.corr()
    s = c.unstack()
    t = s[s != 1]
    return(t.sort_values(ascending = False))

In [ ]:
comment on time

In [ ]:
depar ?!

In [ ]:
https://textblob.readthedocs.io/en/dev/quickstart.html#sentiment-analysis

In [ ]:
https://www.youtube.com/results?search_query=python+text+analysis

In [ ]:
https://github.com/jbesomi/texthero

In [ ]:
https://seaborn.pydata.org/tutorial/color_palettes.html

In [ ]:
https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html
https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a
https://medium.com/@ishan16.d/text-classification-in-python-with-scikit-learn-and-nltk-891aa2d0ac4b
https://nlpforhackers.io/text-classification/
https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/

## 04 : Predictive Modeling
<br>
<span style="color: red;">
<b>Next Steps</b><br>
- Write Demonstrator: Enter a Text, Get a Guess if Recom/Not Recom<br>
</span>

## 05 : Conclusion

### 05-01 : TITLE?

### 05-02 : Future Works
#### 05-02-00 : General Matters
- Contact XING SE and check legal, before publishing<br>

#### 05-02-01 : Data Mining<br>

#### 05-02-02 : Data Cleaning<br>

#### 05-02-03 : Data Analysis<br>
- In Depth Analysis beyond the Recommendation<br>
- Seperate Notebook for Analysis<br>
- Comparing multiple companies<br>
- Text vs No-Text<br>
- Text Lenght<br>

#### 05-02-04 : Predictive Modeling<br>

#### 05-02-05 : Data Visualization<br>
- Automatic generated DINA4 pdf with a Summary Report<br>